# MCMC2: Iterators, Generators, and Channels

Iterators and generators are two important tools in the modern functional programming and Julia implements new "channels." I will show a typical implementation of them for MCMC.

## Iterators

Iterators are often used to save memories. If we wish to do something to each element of a large list, we do not have to store all the elements between the processes. Iterators return each element one by one, so we can do opearations one by one and each time we can discard old data to save memories. Actually, we already used iterators in usual "for" loops.

In [1]:
iter = 1:2

1:2

This is already iterator counting from 1 to 2.

In [2]:
element, state = iterate(iter)

(1, 1)

This is the initial state of the iterator "1:2".

In [3]:
element, state = iterate(iter, state)

(2, 2)

These are the next item and the next state.

In [4]:
result = iterate(iter, state)
result == nothing

true

If there is nothing to iterate, they return nothing.

In [5]:
iter = 1:5
for i in iter
    println(i)
end

1
2
3
4
5


This is the same as follow:

In [6]:
iter_result = iterate(iter)
while iter_result !== nothing
    (i, state) = iter_result
    println(i)
    iter_result = iterate(iter, state)
end

1
2
3
4
5


If we use an iterator instead of a list, we do not need to store the whole list anymore.

In [7]:
list = collect(iter)
for i in 1:length(list)
    println(list[i] ^ 2)
end

1
4
9
16
25


This is meaningless and should be rewritten as:

In [8]:
for i in iter
    println(i ^ 2)
end

1
4
9
16
25


In this way, we can (trivially) avoid allocating a memory for the list. Or this works if the memory does not matter:

In [9]:
for i in list
    println(i ^ 2)
end

1
4
9
16
25


## Generator 1 (generator expression)

An iterator is most useful when combined with a generator. We generally call something creating an iterator a generator. Generators in general include a generator expression, a generator function, etc. Julia supports a generator expression as follows:

In [10]:
gene = (x^2 for x in 1:5)

Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##3#4"))}(getfield(Main, Symbol("##3#4"))(), 1:5)

This is very similar to a list comprehension:

In [11]:
list = [x^2 for x in 1:5]

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

The list comprehension will allocate a memory for the whole list. The generator will save your memory.

In [12]:
for i in gene
    println(i)
end

1
4
9
16
25


## Generator 2 (generator function = resumable function)

An easier, simpler, and most useful way to create an iterator/generator is a generator function, and Python and most modern language support this type of functions. Julia does not natively support Python-type generator functions, but simply you can call ResumableFunctions.jl. This is the fastest way to reproduce the behavior of generator functions.

In [1]:
using ResumableFunctions
@resumable function Ising()::Vector{Int64}
    N = 8
    σ = ones(Int64, N)
    β = 1.0
    while true
        for i in 1 : 1000
            j = rand(1 : N)
            ΔβE = 2β * σ[j] * (σ[mod1(j + 1, N)] + σ[mod1(j - 1, N)])
            -ΔβE > log(rand()) && (σ[j] = -σ[j])
        end
        @yield σ
    end
end

Ising (generic function with 1 method)

This function directely returns an iterater, i.e. a function iterate(iter) is already defined when we call iter = Ising().

In [2]:
iter = Ising()
iter()

8-element Array{Int64,1}:
 -1
 -1
  1
  1
  1
  1
  1
  1

In [3]:
for i in 1 : 10
    println(iter())
end

[-1, -1, -1, -1, -1, -1, -1, -1]
[1, -1, -1, -1, -1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, 1, 1, 1, 1, -1]


ResummableFunctions are very fast and memory-efficient because the macro @resummable will be rewrited to a normal iterator directly before the JIT compilation.

## Channels

Instead of a generator function, Julia natively supports Channel (or Task) for a parallel computing. Especially, Channel is a stronger and more general tool to create an iterator from the function. This is a generalized concept of a generator function, but in most cases ResumableFunctions.jl is faster and memory-efficient, so you do not have to use Channel as the first choice.

In [4]:
using Distributions
function gibbs(a::Float64, b::Float64, c::Float64)::Channel
    Channel(ctype = Tuple{Float64,Float64}) do channel::Channel{Tuple{Float64,Float64}}
        N = 10
        x = 0.0
        y = 0.0
        put!(channel, (x, y))
        for i in 1:N
            x = rand(Normal(b / a * y, 1 / sqrt(a)))
            y = rand(Normal(b / c * x, 1 / sqrt(c)))
            put!(channel, (x, y))
        end
    end
end

gibbs (generic function with 1 method)

Oh, I forgot telling you we can omit "return" in the function!

In [5]:
for z in gibbs(1.0, 0.8, 1.0)
    println(z)
end

(0.0, 0.0)
(0.8111964911327989, 1.2381823954018265)
(1.0514076475579006, 0.9293360859456534)
(0.6585727239187623, 1.1257157575795111)
(1.993100694300263, 3.3922954914269874)
(2.193987085316687, 0.9221132358197086)
(0.6558503695433227, -0.7048515745935189)
(-0.3014385811100516, -0.11981030980940216)
(0.5668510924439532, 0.26267598683102744)
(0.18056466983599723, -1.5240511817381017)
(-0.26136661109383263, 0.970674269268434)


This code is acceptable if the number of tasks is just one.

~ under construction ~

The difference bewteen Channels and ResumableFunctions lies in how they compile the code, and the JIT compiler works better in the case of ResumableFunctions. Check: https://white.ucc.asn.au/2017/11/18/Lazy-Sequences-in-Julia.html